In [69]:
import time
import numpy as np
from keras.applications.vgg16 import VGG16
from keras import backend as k
from keras.preprocessing.image import load_img, img_to_array
from PIL import Image as I
from keras.models import Model
from scipy.optimize import fmin_l_bfgs_b
from scipy.misc import imsave

from IPython.display import Image

In [44]:
height = 512
width = 512

content_image_path = 'instadeep.png'
content_image = I.open(content_image_path)
content_image = content_image.resize((height, width))
content_array = np.asarray(content_image, dtype='float32')
content_array = np.expand_dims(content_array, axis=0)

In [45]:
style_image_path = 'bader_style.png'
style_image = I.open(style_image_path)
style_image = style_image.resize((height, width))
style_array = np.asarray(style_image, dtype='float32')
style_array = np.expand_dims(style_array, axis=0)

In [46]:
content_array[:, :, :, 0] -= 103.939
content_array[:, :, :, 1] -= 116.779
content_array[:, :, :, 2] -= 123.68
content_array = content_array[:, :, :, ::-1]

style_array[:, :, :, 0] -= 103.939
style_array[:, :, :, 1] -= 116.779
style_array[:, :, :, 2] -= 123.68
style_array = style_array[:, :, :, ::-1]

In [47]:
content_image = k.variable(content_array)
style_image = k.variable(style_array)
combination_image = k.placeholder((1, height, width, 3))

In [48]:
input_tensor = k.concatenate([content_image,
                                    style_image,
                                    combination_image], axis=0)

In [50]:
model = VGG16(input_tensor=input_tensor, weights='imagenet',
              include_top=False)

In [52]:
layers = dict([(layer.name, layer.output) for layer in model.layers])

In [53]:
content_weight = 0.025
style_weight = 5.0
total_variation_weight = 1.0

In [54]:
loss = k.variable(0.)

In [55]:
def content_loss(content, combination):
    return k.sum(k.square(combination - content))

layer_features = layers['block2_conv2']
content_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]

loss += content_weight * content_loss(content_image_features,
                                      combination_features)

In [56]:
def gram_matrix(x):
    features = k.batch_flatten(k.permute_dimensions(x, (2, 0, 1)))
    gram = k.dot(features, k.transpose(features))
    return gram

In [57]:
def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = height * width
    return k.sum(k.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

feature_layers = ['block1_conv2', 'block2_conv2',
                  'block3_conv3', 'block4_conv3',
                  'block5_conv3']
for layer_name in feature_layers:
    layer_features = layers[layer_name]
    style_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_features, combination_features)
    loss += (style_weight / len(feature_layers)) * sl

In [58]:
def total_variation_loss(x):
    a = k.square(x[:, :height-1, :width-1, :] - x[:, 1:, :width-1, :])
    b = k.square(x[:, :height-1, :width-1, :] - x[:, :height-1, 1:, :])
    return k.sum(k.pow(a + b, 1.25))

loss += total_variation_weight * total_variation_loss(combination_image)

In [59]:
grads = k.gradients(loss, combination_image)

In [60]:
outputs = [loss]
outputs += grads
f_outputs = k.function([combination_image], outputs)

def eval_loss_and_grads(x):
    x = x.reshape((1, height, width, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    grad_values = outs[1].flatten().astype('float64')
    return loss_value, grad_values

class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

evaluator = Evaluator()

In [65]:
x = np.random.uniform(0, 255, (1, height, width, 3)) - 128.

iterations = 10

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    end_time = time.time()
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
Current loss value: 1.64046e+10
Iteration 0 completed in 265s
Start of iteration 1
Current loss value: 1.26657e+10
Iteration 1 completed in 265s
Start of iteration 2
Current loss value: 1.18358e+10
Iteration 2 completed in 269s
Start of iteration 3
Current loss value: 1.14565e+10
Iteration 3 completed in 264s
Start of iteration 4
Current loss value: 1.12427e+10
Iteration 4 completed in 264s
Start of iteration 5
Current loss value: 1.10744e+10
Iteration 5 completed in 263s
Start of iteration 6
Current loss value: 1.095e+10
Iteration 6 completed in 265s
Start of iteration 7
Current loss value: 1.08574e+10
Iteration 7 completed in 267s
Start of iteration 8
Current loss value: 1.07849e+10
Iteration 8 completed in 268s
Start of iteration 9
Current loss value: 1.0728e+10
Iteration 9 completed in 268s


In [92]:
x = x.reshape((height, width, 3))
x = x[:, :, ::-1]
print(x.shape)
x[:, :, 0] += 103.939
x[:, :, 1] += 116.779
x[:, :, 2] += 123.68
x = np.clip(x, 0, 255).astype('uint8')

I.fromarray(x)

ValueError: cannot reshape array of size 524288 into shape (512,512,3)